# Base Notebook to Run Experiments

## Notebook Setup

### Download Dependencies

In [1]:
!pip install uv
!git clone https://github.com/jorgesilva2407/poc.git /gcr
%cd /gcr
!uv sync --no-dev --no-cache-dir
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 26.9 MB/s eta 0:00:00
Cloning into '/gcr'...
remote: Enumerating objects: 473, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 473 (delta 0), reused 0 (delta 0), pack-reused 472 (from 1)
Receiving objects: 100% (473/473), 70.46 MiB | 19.57 MiB/s, done.
Resolving deltas: 100% (228/228), done.
/gcr
Using CPython 3.11.14
Creating virtual environment at: .venv
Resolved 186 packages in 0.87ms
Prepared 94 packages in 1m 17s
Installed 94 packages in 939ms
 + absl-py==2.3.1
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.0
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + anyio==4.11.0
 + attrs==25.4.0
 + cachetools==6.2.1
 + certifi==2025.10.5
 + charset-normalizer==3.4.3
 + decorator==5.2.1
 + docstring-parser==0.17.0
 + filelock==3.20.0
 + frozenlist==1.8.0
 + fsspec==2025.9.0
 + gcsfs==2025.9.0
 + google-api-core==2.27.0
 + google-auth==2.41.1
 + google-auth-oauthlib==1.2.2
 + google-cloud-aiplatform==1.124.0
 + google-clo

### Import Depedencies
- Import dependencies
- Mount Google Drive
- Define global variables
    - Assert paths exist

In [2]:
import os
import json
import shlex
from enum import Enum
from pathlib import Path
from dataclasses import dataclass

import optuna
from google.colab import drive

drive.mount("/content/drive")

TENSORBOARD_LOG_DIR = "/content/drive/MyDrive/poc/tensorboard"
ARTIFACTS_DIR = "/content/drive/MyDrive/poc/artifacts"
METRIC_FILE = "/tmp/metric.json"
DATA_DIR = "/content/drive/MyDrive/poc/data"
OPTUNA_DIR = "/content/drive/MyDrive/poc/optuna"

Mounted at /content/drive


## Optuna Setup

### Define Hyper Parameter Types

In [3]:
class HParamType(Enum):
    LOGUNIFORM = "loguniform"
    UNIFORM = "uniform"
    INTEGER = "integer"
    CATEGORICAL = "categorical"


@dataclass
class HParam:
    type: HParamType

    def suggest(self, trial, name: str):
        raise NotImplementedError


@dataclass
class Categorical(HParam):
    choices: list[int | float | str]

    def __init__(self, choices: list[int | float | str]):
        super().__init__(HParamType.CATEGORICAL)
        self.choices = choices

    def suggest(self, trial, name: str):
        return trial.suggest_categorical(name, self.choices)


@dataclass
class Uniform(HParam):
    low: float
    high: float

    def __init__(self, low: float, high: float):
        super().__init__(HParamType.UNIFORM)
        self.low = low
        self.high = high

    def suggest(self, trial, name: str):
        return trial.suggest_uniform(name, self.low, self.high)


@dataclass
class LogUniform(HParam):
    low: float
    high: float

    def __init__(self, low: float, high: float):
        super().__init__(HParamType.LOGUNIFORM)
        self.low = low
        self.high = high

    def suggest(self, trial, name: str):
        return trial.suggest_loguniform(name, self.low, self.high)


@dataclass
class Integer(HParam):
    low: int
    high: int

    def __init__(self, low: int, high: int):
        super().__init__(HParamType.INTEGER)
        self.low = low
        self.high = high

    def suggest(self, trial, name: str):
        return trial.suggest_int(name, self.low, self.high)

### Helper Function to get a set of Hyper Parameters to use

In [4]:
def suggest_params(trial, search_space):
    params = {}
    for name, hparam in search_space.items():
        params[name] = hparam.suggest(trial, name)
    return params

### Model Runner

In [5]:
def run_model(
    model_name: str,
    params: dict,
    dataset: str,
    all_csv: str,
    train_csv: str,
    val_csv: str,
    test_csv: str,
) -> float:
    tensorboard_log_dir = Path(TENSORBOARD_LOG_DIR) / dataset
    artifacts_dir = Path(ARTIFACTS_DIR) / dataset

    os.makedirs(tensorboard_log_dir, exist_ok=True)
    os.makedirs(artifacts_dir, exist_ok=True)

    cmd = [
        "uv",
        "run",
        "python",
        "-u",
        "main.py",
        "--model",
        model_name,
        "--logger",
        "TensorBoard",
        "--tensorboard-log-dir",
        str(tensorboard_log_dir),
        "--artifacts-saver",
        "Local",
        "--optuna-metric-file",
        METRIC_FILE,
        "--local-artifacts-path",
        str(artifacts_dir),
        "--experiment-tracker",
        "Optuna",
        "--batch-size",
        64,
        "--all-interactions-csv",
        all_csv,
        "--train-interactions-csv",
        train_csv,
        "--validation-interactions-csv",
        val_csv,
        "--test-interactions-csv",
        test_csv,
    ]

    for param, value in params.items():
        cmd.extend([param, str(value)])

    cmd = [str(c) for c in cmd]

    shell_cmd = shlex.join(cmd)
    print(shell_cmd)

    !{shell_cmd}

    metric = None
    with open(METRIC_FILE, "r") as f:
        metric = json.load(f)
    return metric["NDCG@5"]

### Define the Objective of the Optuna Study

In [6]:
def objective_factory(model_name: str, search_space: dict, dataset: str):
    def objective(trial):
        params = suggest_params(trial, search_space)

        data_dir = Path(DATA_DIR) / dataset
        all_csv = data_dir / "all_interactions.csv"
        train_csv = data_dir / "split" / "train.csv"
        val_csv = data_dir / "split" / "val.csv"
        test_csv = data_dir / "split" / "test_neg_samples.csv"

        paths = [data_dir, all_csv, train_csv, val_csv, test_csv]
        missing_paths = [path for path in paths if not path.exists()]

        if missing_paths:
            raise ValueError(f"Missing paths: {missing_paths}")

        result = run_model(
            model_name, params, dataset, all_csv, train_csv, val_csv, test_csv
        )
        return result

    return objective

### Run an Optuna Study

In [7]:
def optimize_model(
    model_name: str, search_space: dict, dataset: str, n_trials: int = 20
):
    db_path = Path(OPTUNA_DIR) / dataset / f"{model_name}.db"
    os.makedirs(db_path.parent, exist_ok=True)

    study = optuna.create_study(
        study_name=f"{model_name}Optimization",
        direction="maximize",
        storage=f"sqlite:///{db_path}",
        load_if_exists=True,
        sampler=optuna.samplers.TPESampler(seed=42),
    )

    objective = objective_factory(model_name, search_space, dataset)
    study.optimize(objective, n_trials=n_trials)

    print("\n✅ Optimization complete!")
    print(f"Best value: {study.best_value}")
    print(f"Best params: {study.best_params}")

    return study

## Run the Study

Update the model, dataset and search space as needed

In [8]:
model = "BiasedSVD"
dataset = "beauty"

embedding_dims = [16, 32, 64, 128, 256]
learning_rates = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 1e-2]
weight_decays = [1e-6, 1e-5, 1e-4, 1e-3]

search_space = {
    "--embedding-dim": Categorical(choices=embedding_dims),
    "--learning-rate": Categorical(choices=learning_rates),
    "--weight-decay": Categorical(choices=weight_decays),
}

In [ ]:
optimize_model(model, search_space, dataset, n_trials=20)

[I 2025-11-11 02:00:58,548] A new study created in RDB with name: BiasedSVDOptimization


uv run python -u main.py --model BiasedSVD --logger TensorBoard --tensorboard-log-dir /content/drive/MyDrive/poc/tensorboard/beauty --artifacts-saver Local --optuna-metric-file /tmp/metric.json --local-artifacts-path /content/drive/MyDrive/poc/artifacts/beauty --experiment-tracker Optuna --batch-size 64 --all-interactions-csv /content/drive/MyDrive/poc/data/beauty/all_interactions.csv --train-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/train.csv --validation-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/val.csv --test-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/test_neg_samples.csv --embedding-dim 32 --learning-rate 0.0001 --weight-decay 1e-06
Installed 88 packages in 373ms
Ignored arguments: []
Using device: cuda
Epoch 1 - Training: 100% 350/350 [00:34<00:00, 10.20it/s]
Epoch 1 - Validation: 100% 35/35 [00:03<00:00, 10.41it/s]
Epoch 1: Train Loss = 3.2444, Val Loss = 3.2034, Val NDCG@5 = 0.0328, Val NDCG@10 = 0.0516
Epoch 2 - Trainin

[I 2025-11-11 02:12:36,900] Trial 0 finished with value: 0.028768496587872505 and parameters: {'--embedding-dim': 32, '--learning-rate': 0.0001, '--weight-decay': 1e-06}. Best is trial 0 with value: 0.028768496587872505.


uv run python -u main.py --model BiasedSVD --logger TensorBoard --tensorboard-log-dir /content/drive/MyDrive/poc/tensorboard/beauty --artifacts-saver Local --optuna-metric-file /tmp/metric.json --local-artifacts-path /content/drive/MyDrive/poc/artifacts/beauty --experiment-tracker Optuna --batch-size 64 --all-interactions-csv /content/drive/MyDrive/poc/data/beauty/all_interactions.csv --train-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/train.csv --validation-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/val.csv --test-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/test_neg_samples.csv --embedding-dim 64 --learning-rate 0.01 --weight-decay 0.0001
Ignored arguments: []
Using device: cuda
Epoch 1 - Training: 100% 350/350 [00:35<00:00,  9.74it/s]
Epoch 1 - Validation: 100% 35/35 [00:03<00:00, 10.82it/s]
Epoch 1: Train Loss = 1.3398, Val Loss = 0.7763, Val NDCG@5 = 0.0522, Val NDCG@10 = 0.0772
Epoch 2 - Training: 100% 350/350 [00:35<00:00,  9

[I 2025-11-11 02:25:41,552] Trial 1 finished with value: 0.11725813895463943 and parameters: {'--embedding-dim': 64, '--learning-rate': 0.01, '--weight-decay': 0.0001}. Best is trial 1 with value: 0.11725813895463943.


...

uv run python -u main.py --model BiasedSVD --logger TensorBoard --tensorboard-log-dir /content/drive/MyDrive/poc/tensorboard/beauty --artifacts-saver Local --optuna-metric-file /tmp/metric.json --local-artifacts-path /content/drive/MyDrive/poc/artifacts/beauty --experiment-tracker Optuna --batch-size 64 --all-interactions-csv /content/drive/MyDrive/poc/data/beauty/all_interactions.csv --train-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/train.csv --validation-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/val.csv --test-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/test_neg_samples.csv --embedding-dim 64 --learning-rate 0.0005 --weight-decay 1e-06
Ignored arguments: []
Using device: cuda
Epoch 1 - Training: 100% 350/350 [00:34<00:00, 10.14it/s]
Epoch 1 - Validation: 100% 35/35 [00:04<00:00,  7.11it/s]
Epoch 1: Train Loss = 4.1510, Val Loss = 4.0068, Val NDCG@5 = 0.0279, Val NDCG@10 = 0.0441
Epoch 2 - Training: 100% 350/350 [00:34<00:00, 1

[I 2025-11-11 08:41:02,210] Trial 18 finished with value: 0.030433297157287598 and parameters: {'--embedding-dim': 64, '--learning-rate': 0.0005, '--weight-decay': 1e-06}. Best is trial 4 with value: 0.11771607398986816.


uv run python -u main.py --model BiasedSVD --logger TensorBoard --tensorboard-log-dir /content/drive/MyDrive/poc/tensorboard/beauty --artifacts-saver Local --optuna-metric-file /tmp/metric.json --local-artifacts-path /content/drive/MyDrive/poc/artifacts/beauty --experiment-tracker Optuna --batch-size 64 --all-interactions-csv /content/drive/MyDrive/poc/data/beauty/all_interactions.csv --train-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/train.csv --validation-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/val.csv --test-interactions-csv /content/drive/MyDrive/poc/data/beauty/split/test_neg_samples.csv --embedding-dim 64 --learning-rate 5e-05 --weight-decay 0.0001
Ignored arguments: []
Using device: cuda
Epoch 1 - Training: 100% 350/350 [00:36<00:00,  9.68it/s]
Epoch 1 - Validation: 100% 35/35 [00:03<00:00, 10.62it/s]
Epoch 1: Train Loss = 4.5133, Val Loss = 4.4978, Val NDCG@5 = 0.0272, Val NDCG@10 = 0.0421
Epoch 2 - Training: 100% 350/350 [00:35<00:00,  

[I 2025-11-11 08:56:46,799] Trial 19 finished with value: 0.030011828988790512 and parameters: {'--embedding-dim': 64, '--learning-rate': 5e-05, '--weight-decay': 0.0001}. Best is trial 4 with value: 0.11771607398986816.



✅ Optimization complete!
Best value: 0.11771607398986816
Best params: {'--embedding-dim': 64, '--learning-rate': 0.001, '--weight-decay': 0.0001}
